<a href="https://colab.research.google.com/github/shahhet14/MNIST_Digit_Recognizer/blob/main/MNIST_digit_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [59]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
train_y = train['label'].astype('int32')
train_x = train.drop(['label'], axis=1).astype('float32')
test_x = test.astype('float32')
train_y.shape, train_x.shape, test_x.shape

((42000,), (42000, 784), (28000, 784))

In [62]:
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x/255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x/255.0
train_x.shape, test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [63]:
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [64]:
print(train['label'].head())
print(train_y[0:5,:])

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [65]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

In [66]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

(33600, 28, 28, 1)
(33600, 10)
(8400, 28, 28, 1)
(8400, 10)


In [79]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.0001, 
    patience=5, 
    restore_best_weights=True,
)

In [80]:
cnn_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28,28,1)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.08),

    tf.keras.layers.Conv2D(filters=24, kernel_size=(3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(filters=36, kernel_size=(3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(filters=48, kernel_size=(3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
cnn_model.build()
cnn_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_rotation_3 (RandomRot (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 24)        240       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 24)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 24)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 36)        7812      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 7, 7, 36)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 36)         

In [81]:
cnn_model.compile(
    optimizer = tf.keras.optimizers.Adam( learning_rate=0.001),
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [82]:
cnn_model.fit(
    train_x, 
    train_y,
    batch_size=64,
    epochs=50,
    validation_data=(val_x,val_y),
    callbacks=[early_stopping]
    )

Epoch 1/50
525/525 [==============================] - 3s 5ms/step - loss: 1.3554 - accuracy: 0.5143 - val_loss: 0.1177 - val_accuracy: 0.9663
Epoch 2/50
525/525 [==============================] - 2s 4ms/step - loss: 0.2836 - accuracy: 0.9125 - val_loss: 0.0734 - val_accuracy: 0.9770
Epoch 3/50
525/525 [==============================] - 3s 5ms/step - loss: 0.1989 - accuracy: 0.9394 - val_loss: 0.0555 - val_accuracy: 0.9820
Epoch 4/50
525/525 [==============================] - 2s 5ms/step - loss: 0.1701 - accuracy: 0.9496 - val_loss: 0.0529 - val_accuracy: 0.9818
Epoch 5/50
525/525 [==============================] - 2s 4ms/step - loss: 0.1477 - accuracy: 0.9557 - val_loss: 0.0502 - val_accuracy: 0.9843
Epoch 6/50
525/525 [==============================] - 2s 4ms/step - loss: 0.1262 - accuracy: 0.9613 - val_loss: 0.0458 - val_accuracy: 0.9855
Epoch 7/50
525/525 [==============================] - 2s 5ms/step - loss: 0.1247 - accuracy: 0.9609 - val_loss: 0.0419 - val_accuracy: 0.9868
Epoch 

In [83]:
results = cnn_model.predict(test_x)
results = np.argmax(results, axis=1)
results = pd.Series(results,name='Label')

In [72]:
submission = pd.concat([pd.Series(range(1,28001), name="ImageId"),results],axis=1)

In [84]:
submission.to_csv("submission10.csv",index=False)

In [85]:
def recognize_digit(image):
    image = np.array([image])
    prediction = cnn_model.predict(image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

In [75]:
recognize_digit(test_x[1])

{'0': 0.9999946355819702,
 '1': 9.24736884289068e-11,
 '2': 2.682445483515039e-06,
 '3': 5.85269654962417e-09,
 '4': 5.484180665149552e-09,
 '5': 5.439896710868197e-08,
 '6': 2.1508797090064036e-06,
 '7': 2.5753708499109962e-08,
 '8': 7.254477907281398e-08,
 '9': 2.440956450300291e-07}

In [ ]:
!pip install gradio

In [86]:
import gradio as gr

In [87]:
sketch_in = gr.inputs.Image(shape=(28, 28), image_mode='L', invert_colors=False, source="canvas")
label_out = gr.outputs.Label(num_top_classes=3)

iface = gr.Interface(
    recognize_digit, 
    sketch_in, 
    label_out,
    interpretation="default",
    capture_session=True,
    title="Digit Recognizer Trained on MNIST dataset",
    description="Draw a digit (0-9) on the sketchpad and click SUBMIT to see if the model can recognize it!"
)

iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://56686.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7871/',
 'https://56686.gradio.app')